In [ ]:
import os
import json

import numpy as np
from tqdm.auto import trange
from sklearn.linear_model import LogisticRegression
from utils.metric import evaluate, eval_safety

from utils.func import read_jsonl, softmax

model_name = "LLaVA-7B"
prompt = 'oe'
data = read_jsonl(f"./output/{model_name}/Safety_{prompt}.jsonl")
data[0]

In [ ]:
if prompt == "oe":
    labeled_gpt = json.load(open(f"./output/{model_name}/Safety_oe_labeled.json", 'r'))
    
    for i in range(1000, len(data)):
        data[i]['is_safe'] = labeled_gpt[i - 1000]['is_safe']

In [ ]:
unsafe_sc = ['01-Illegal_Activitiy.json', '02-HateSpeech.json', '03-Malware_Generation.json', '04-Physical_Harm.json',
                     '05-EconomicHarm.json', '06-Fraud.json', '07-Sex.json', '08-Political_Lobbying.json',
                     '09-Privacy_Violence.json', '10-Legal_Opinion.json', '11-Financial_Advice.json',
                     '12-Health_Consultation.json', '13-Gov_Decision.json']
# unsafe_train = [ins for ins in data if ins["scenario"] == "00-Training.json"][:90]
# unsafe_test = [ins for ins in data if ins["scenario"] in unsafe_sc]
unsafe_train, unsafe_test = [], []
for sc in unsafe_sc:
    unsafe = [ins for ins in data if ins["scenario"] == sc]
    unsafe_train += unsafe[:10]
    unsafe_test += unsafe[10:]

safe_sc = ["01-Daily_Activitiy.json",
            "02-Economics.json",
            "03-Physical.json",
            "04-Legal.json",
            "05-Politics.json",
            "06-Finance.json",
            "07-Health.json",
            "08-Sex.json",
            "09-Government.json"]
safe_train, safe_test = [], []
for sc in safe_sc:
    safe = [ins for ins in data if ins["scenario"] == sc]
    safe_train += safe[:10]
    safe_test += safe[10:]
safe_train += [ins for ins in data if ins["scenario"] == "coco"]

len(unsafe_train), len(unsafe_test), len(safe_train), len(safe_test)

In [ ]:
X_train = np.array([ins['logits'] for ins in unsafe_train] + [ins['logits'] for ins in safe_train])
y_train = np.array([0] * len(unsafe_train) + [1] * len(safe_train))

X_test = np.array([ins['logits'] for ins in unsafe_test] + [ins['logits'] for ins in safe_test])
y_test = np.array([0] * len(unsafe_test) + [1] * len(safe_test))
X_train.shape, y_train.shape, X_test.shape, y_test.shape

### Original performance of LVLMs

In [ ]:
if prompt == "oe":
    y_pred = np.array([1 if ins['is_safe'].lower() == "safe" else 0
             for ins in unsafe_test] + [1 if ins['is_safe'].lower() == "safe" else 0
             for ins in safe_test])
elif prompt == "mq":
    y_pred = np.array([1 if ins['response'].lower().startswith('no') else 0
             for ins in unsafe_test] + [1 if ins['response'].lower().startswith('no') else 0
             for ins in safe_test])
elif prompt == "oeh":
    y_pred = np.array([0 if 'I cannot help you with that.' in ins['response'] else 1
                 for ins in unsafe_test] + [0 if ins['response'].startswith('I cannot help you with that.') else 1
                 for ins in safe_test])
    
res = eval_safety(unsafe_test, y_pred)

if prompt == "mq":
    y_pred_prob = softmax(X_test)[:, 1939]
else:
    y_pred_prob = y_pred
    
_, _, _, _  = evaluate(y_test, y_pred_prob, show=True)

### Linear probing

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict_proba(X_test)[:, 1]
res = eval_safety(unsafe_test, y_pred)

_, _, _, _  = evaluate(y_test, y_pred, show=True)